# Checkers

## Regole della dama
Da: https://www.federdama.org/cms/index.php/documenti-1/le-specialita/dama-italiana
- la damiera è di dimensione 8 X 8 e l'ultima casella in basso deve essere di colore nero
- ogni giocatore ha 12 pedine a testa
- le pedine si muovono sempre in diagonale sulle case scure, di una casa alla volta e sempre in avanti
- quando una pedina raggiunge l'altra estremità della scacchiera diventa dama
- ogni pedina *è obbligata* a mangiare quelle avversarie che si trovano davanti nella casella diagonale se la casella successiva è libera. Se ci sono ulteriori pedine e le caselle diagonali sono libere, si continua a mangiare (fino a 3 pezzi). Bisogna mangiare obbligatoriamente il maggior numero di pezzi
- è obbligatorio mangiare con il pezzo più forte (dama)

Le parti di codice da implementare sono indicate dalla dicitura `# TODO` e sono:
- algoritmo min-max
- algoritmo alpha-beta
- calcolo dell'utility

La profondità dell'albero di ricerca può essere modificata cambiando il valore
```
self.engine = Engine(3)
```
che si trova nel metodo `__init__` della classe `UI`. Nell'esempio sopra, la profondità è fissata a 3. Per fissarla a 2, per esempio, basterà sostituire il 3 con il 2. Notare: con profondità maggiore di 4 l'algoritmo rallenta molto.

Installare le librerie necessarie. Da terminale

In [42]:
# $ pip install pygame

Importiamo le librerie necessarie

In [1]:
import pygame
from pygame.locals import *
import random
import copy
from collections import namedtuple
from typing import Union

pygame 2.4.0 (SDL 2.26.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


Definizione di alcune costanti per la visualizzazione della scacchiera.

In [2]:
BLACK = (0,   0,   0)
BLACK_WOOD = (141, 76, 32)
DARK_CHECKER = (72, 32, 0)
DARK_CHECKER_BORDER = (62, 32, 0)
LIGHT_CHECKER = (230, 218, 151)
LIGHT_CHECKER_BORDER = (210, 198, 131)
WHITE = (255, 255, 255)
WHITE_WOOD = (251, 229, 200)
GREEN = (0, 255,   0)
GREEN_DARK = (0, 153, 51)
RED = (255,   0,   0)
RED_LIGHT = (255, 77, 77)
BLUE = (0,   0, 255)
BLUE_DARK = (77, 77, 255)
YELLOW = (255, 255,   0)
TRANS = (1,   2,   3)

WIDTH = 700
HEIGHT = 700
MARK_SIZE = 50

Definiamo la scacchiera e lo stato come delle `namedtuple`. Vedere: https://realpython.com/python-namedtuple/.

In [3]:
Board = namedtuple('Board', 'cells')
State = namedtuple('State', 'board, to_move')

La scacchiera è rappresentata come una matrice (lista di liste) dove ciascuna entry è:

- `'x'` oppure `'o'` dove `'x'` è una pedina del giocatore nero e `'o'` una pedina del giocatore bianco
- `'-'` è una casella libera
- `'X'` e `'O'` (maiuscole) rappresentano le dame

La cella in alto a sinistra ha coordinate 0,0 mentre quella in basso a destra 7,7.

Lo stato del gioco è rappresentato tramite una coppia `(board, to_move)` dove `board` è una scacchiera come indicato sopra e `to_move` è `'x'` oppure `'o'` e rappresenta il giocatore che deve muovere.

### Classe Game

Definiamo una classe `Game` che rappresenta un gioco.

In [4]:
class Game:
    
    # metodo che controlla se lo stato corrente è terminale
    def is_terminal(self, state) -> bool:
        return self.actions(state) == []

    
    # metodo che restituisce il giocatore che deve muovere
    def to_move(self, state):
        return state.to_move

    
    '''
    Metodo che definisce l'utilità di uno stato dato un giocatore.
    Questo metodo viene utilizzato nell'algoritmo min-max e alpha-beta.
    Rappresentare l'utilità dello stato come si preferisce (ad esempio:
    numero di pedine che restano al giocatore tenendo conto che una dama
    è più importante di una pedina).
    '''
    def utility(self, state, player):
        # TODO: provare ad implementare altre utility
        mine = 0
        for row in state.board:
            for cell in row:
                if cell == player:
                    mine = mine + 1
                if cell == player.upper():
                    mine = mine + 2
                if cell == opponent(player):
                    mine = mine - 1
                if cell == opponent(player).upper():
                    mine = mine - 2
        return mine


    '''
    Metodo che restituisce una lista di azioni possibili dato uno stato.
    La azioni possibili sono rappresentate tramite una lista di tuple.
    Ci sono due possibilità:
    - spostamento pedina o dama ('move', (row, col), (delta_row, delta_col))
    - salto pedina o dama ('jump', (row, col), (delta_row, delta_col))
    Prima controllo se ci sono mosse possibili di salto perché se posso
    mangiare una pedina devo farlo; se non ci sono mosse di salto allora
    controllo le mosse di spostamento.
    
    Esempio 1: [('jump', (3, 5), [(2, 2)])]
    
    Esempio 2: [('move', (5, 1), (-1, -1)), ('move', (5, 1), (-1, 1)), 
    ('move', (5, 3), (-1, -1)), ('move', (5, 3), (-1, 1)), 
    ('move', (5, 5), (-1, -1)), ('move', (5, 5), (-1, 1))]
    
    Questo metodo invoca la funzione jumpingchains (o can_jump) e
    la funzione can_move, che sono definite più avanti. 
    '''
    def actions(self, state):
        moves = []  # inizialmente la lista delle azioni è vuota
        for row in range(8):  # per ogni riga
            for col in range(8):  # per ogni colonna
                if state.board[row][col].lower() == state.to_move:  # se sono il giocatore che deve muovere 
                    for chain in jumpingchains(state.board, (row, col)):  # per ogni elemento della jumpingchains
                        moves.append(('jump', (row, col), chain))  # aggiungo l'elemento nella lista di azioni possibili
        if moves == []:  # moves è vuota ([]) solamente se non ho nessuna pedina da mangiare. In questo caso, posso spostarmi
            for row in range(8):  # per ogni riga
                for col in range(8):  # per ogni colonna
                    if state.board[row][col].lower() == state.to_move:  # se sono il giocatore che deve muovere
                        for delta_row in [-1, 1]:
                            for delta_col in [-1, 1]:
                                # ciclo per ogni coppia (-1,-1), (-1,1), (1,-1), (1,1)
                                if can_move(state.board, (row, col), (delta_row, delta_col)): 
                                    # se posso spostarmi nella direzione selezionata
                                    moves.append(('move', (row, col), (delta_row, delta_col)))
                                    # aggiungo un'azione alla lista
        
        return moves # restituisco le azioni possibili

    
    # metodo che controlla se una mossa (action) è valida dato uno stato (state)
    def is_valid_move(self, state, action) -> bool:
        actions = self.actions(state)
        return action in actions
    
    
    # metodo che applica allo stato corrente (state) una azione (action) e
    # restituisce il nuovo stato aggiornato
    def result(self, state, action):
        type, (from_row, from_col), movements = action
        if type == 'move':
            token_char = state.board[from_row][from_col]
            to_row = from_row+movements[0]
            to_col = from_col+movements[1]
            if token_char == 'x' and to_row == 7:
                token_char = 'X'
            if token_char == 'o' and to_row == 0:
                token_char = 'O'
            board = copy.deepcopy(state.board)
            board[to_row][to_col] = token_char
            board[from_row][from_col] = '-'
        elif type == 'jump':
            start = from_row, from_col
            board = copy.deepcopy(state.board)
            for m in movements:
                start, board = jump(board, start, m)
        state = State(board, opponent(state.to_move))
        return state

Definiamo una classe `Engine` che rappresenta un motore di ricerca nello spazio degli stati. Nel codice seguente viene selezionata una mossa casuale. Le linee di codice per min-max o alpha-beta sono commentate. Quando i metodi saranno implementati, basterà togliere il commento dall'algoritmo da provare.

In [16]:
class Engine:
    # max_depth è la profondità massima di esplorazione
    def __init__(self, max_depth):
        self.max_depth = max_depth
    
    
    # metodo che restituisce la mossa migliore dato uno stato (state)
    def best_move(self, state):
        game = Game()
        
        # algoritmo min-max
        # return minmax_search(state,game,self.max_depth)
        
        # algoritmo alpha-beta
        # return alpha_beta_search(state,game,self.max_depth)
        
        # scelta mossa casuale
        # print(game.to_move(state))
        return random.choice(game.actions(state))

### Algoritmo min-max

Suggerimento: implementare prima min-max e poi alpha-beta.

Alcune funzioni utili in python: [`max(a,b)`](https://docs.python.org/3/library/functions.html#max) (in generale può ricevere altri parametri) che restituisce il più grande tra `a` e `b`.
Esempio:
```
>>> max(3,4)
4
```
Analogamente, [`min(a,b)`](https://docs.python.org/3/library/functions.html#min) restituisce il più piccolo tra `a` e `b`.
Esempio:
```
>>> min(3,4)
3
```
Utilizzare eventualmente max con funzioni lambda (non indispensabile).

In [6]:
def minmax_search(state, game, depth):
    '''
    Metodo che implementa la funzione min-max.
    Suggerimento: devono essere utilizzati i metodi:
    - game.is_terminal: controlla che lo stato sia terminale
    - game.utility: restituisce l'utilità di uno stato per un giocatore
    - game.actions: restituisce la lista di azioni possibili in uno stato
    - game.result: restituisce il risultato dell'applicazione di un'azione in uno stato
    '''

    def max_value(state, depth, player):
        # print(f"player: {player}, depth: {depth}")
        if depth == 0 or game.is_terminal(state):
            return game.utility(game, state, player)
        v = -1000
        actions = game.actions(state)
        for a in actions:
            v = max(v, min_value(game.result(state, a), depth - 1, player))
        return v

    def min_value(state, depth, player):
        # print(f"player: {player}, depth: {depth}")
        if depth == 0 or game.is_terminal(state):
            return game.utility(game, state, player)
        v = 1000
        actions = game.actions(state)
        for a in actions:
            v = min(v, max_value(game.result(state, a), depth - 1, player))
        return v

    # body di minmax_search:
    best = max(game.actions(state), key=lambda a: min_value(game.result(state, a), depth, game.to_move(state)))

    return best


### Algoritmo alpha-beta

Suggerimento: implementare prima min-max e poi alpha-beta.

In [7]:
def alpha_beta_search(state, game, depth):
    '''
    Metodo che implementa la funzione alpha-beta.
    Suggerimento: devono essere utilizzati i metodi:
    - game.is_terminal: controlla che lo stato sia terminale
    - game.utility: restituisce l'utilità di uno stato per un giocatore
    - game.actions: restituisce la lista di azioni possibili in uno stato
    - game.result: restituisce il risultato dell'applicazione di un'azione in uno stato
    '''

    def max_value(state, depth, alpha, beta, player):
        if depth == 0 or game.is_terminal(state):
            return game.utility(state, player)
        v = -1000
        actions = game.actions(state)

        for a in actions:
            v = max(v, min_value(game.result(state, a), depth - 1, alpha, beta, player))
            if v >= beta:
                return v
            alpha = max(alpha,v)
        return v

    def min_value(state, depth, alpha, beta, player):
        if depth == 0 or game.is_terminal(state):
            return game.utility(state, player)
        v = 1000
        actions = game.actions(state)

        for a in actions:
            v = min(v, max_value(game.result(state, a), depth - 1, alpha, beta, player))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body di alpha_beta_pruning
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), depth, -1000, 1000, game.to_move(state)))

### Interfaccia Grafica

Definiamo una classe `UI` che reppresenta l'interfaccia.

In [8]:
class UI:
    def __init__(self):
        self.game = Game()
        self.engine = Engine(3)  # <----- massima profondità 3
        self.engine_next_move = None
        self.turn = random.randrange(2) # il giocatore iniziale è random
        self.players = ['x', 'o']
        self.winner = None
        self.selected_token = None
        self.is_game_over = False
        self.game_over_message = None
        pygame.display.set_caption("Turno di %s" % ("nero" if self.players[self.turn % 2] == 'x' else "bianco"))
        self.game_board = [['x', '-', 'x', '-', 'x', '-', 'x', '-'],
                           ['-', 'x', '-', 'x', '-', 'x', '-', 'x'],
                           ['x', '-', 'x', '-', 'x', '-', 'x', '-'],
                           ['-', '-', '-', '-', '-', '-', '-', '-'],
                           ['-', '-', '-', '-', '-', '-', '-', '-'],
                           ['-', 'o', '-', 'o', '-', 'o', '-', 'o'],
                           ['o', '-', 'o', '-', 'o', '-', 'o', '-'],
                           ['-', 'o', '-', 'o', '-', 'o', '-', 'o']]
        self.clicks = []
        self.state = State(self.game_board, self.players[self.turn % 2])
        self.actions = self.game.actions(self.state)

        
    def evaluate_click(self, mouse_pos, keys):
        # seleziona un token se nessuno è attualmente selezionato
        # sposta il token in un quadrato se è una mossa valida

        self.engine_next_move = None

        row, col = get_clicked_row(mouse_pos), get_clicked_column(mouse_pos)
        if self.selected_token:
            self.clicks.append((row, col))
            if not (keys[pygame.K_RSHIFT] or keys[pygame.K_LSHIFT]):
                self.play(self.selected_token, self.clicks)
            elif row == self.selected_token[0] and col == self.selected_token[1]:
                self.selected_token = None
                self.clicks = []
            else:
                print('Mossa non valida')
        else:
            if self.game_board[row][col].lower() == self.players[self.turn % 2]:
                self.selected_token = [row, col]
                self.clicks = []

                
    def play(self, token_location, clicks):
        # sposta il token in una posizione
        from_row, from_col = token_location
        movements = [(clicks[0][0] - from_row, clicks[0][1] - from_col)]
        for i in range(1, len(clicks)):
            movements.append(
                (clicks[i][0]-clicks[i-1][0], clicks[i][1]-clicks[i-1][1]))
        if abs(movements[0][0]) == 1:
            move = ('move', (from_row, from_col), movements[0])
        else:
            move = ('jump', (from_row, from_col), movements)
        state = State(self.game_board,
                      self.game_board[from_row][from_col].lower())
        if self.game.is_valid_move(state, move):
            self.game_board = self.game.result(state, move).board
            self.next_turn()
        else:
            self.selected_token = None
            self.clicks = []

            
    def engine_move(self):
        # diviso in due parti
        # - calcola la mossa con il motore
        # - esegui la mossa 
        if self.engine_next_move:
            self.game_board = self.game.result(self.state, self.engine_next_move).board
            self.engine_next_move = None
            self.next_turn()
        else:
            engine_action = self.engine.best_move(self.state)
            print(engine_action)
            (_, (row, col), _) = engine_action
            self.selected_token = [row, col]
            self.engine_next_move = engine_action

            
    def next_turn(self):
        self.selected_token = None
        self.turn += 1
        self.state = State(self.game_board, self.players[self.turn % 2])
        self.actions = self.game.actions(self.state)
        if not self.actions:
            self.is_game_over = True
            pygame.display.set_caption("Game Over")
            if self.players[self.turn % 2] == 'x':
                message = "Bianco"
                self.winner = 'o'
            else:
                message = "Nero"
                self.winner = 'x'
            self.game_over_message = message + " ha vinto"
        else:
            pygame.display.set_caption("Turno: %s" % ("nero" if self.players[self.turn % 2] == 'x' else "bianco"))

            
    def draw(self):
        # disegna scacchiera
        for i in range(9):
            for j in range(9):
                if (i + j) % 2 != 0:
                    pos_left = i * (WIDTH / 8)
                    pos_top = j * (HEIGHT / 8)
                    pygame.draw.rect(screen, WHITE_WOOD, ((pos_left, pos_top), (WIDTH / 8, HEIGHT / 8)))

        font = pygame.font.SysFont('Calibri', MARK_SIZE, False, False)
        font.set_bold(True)
        
        # visualizza il messaggio di fine partita
        if self.is_game_over:
            text = font.render(self.game_over_message, True, BLACK)
            pygame.draw.rect(screen, WHITE, (
                (WIDTH / 2 - text.get_width() / 2 - 10, HEIGHT / 2 - text.get_height() / 2 - 10),
                (text.get_width() + 20, text.get_height() + 20)))
            screen.blit(
                 text, [WIDTH / 2 - text.get_width() / 2, HEIGHT / 2 - text.get_height() / 2])
            return
        
        # evidenzia le mosse possibili
        self.highlight_actions(self.actions)

        # disegna le pedine sulla scacchiera
        for r in range(len(self.game_board)):
            for c in range(len(self.game_board[r])):
                mark = self.game_board[r][c]
                if mark != '-':
                    x = WIDTH / 8 * c + WIDTH / 16
                    y = HEIGHT / 8 * r + HEIGHT / 16
                    if mark == 'x':
                        pygame.draw.circle(screen, DARK_CHECKER, (x, y), WIDTH / 24)
                        pygame.draw.circle(screen, DARK_CHECKER_BORDER, (x, y), WIDTH / 24, 5)
                    elif mark == 'o':
                        pygame.draw.circle(screen, LIGHT_CHECKER, (x, y), WIDTH / 24)
                        pygame.draw.circle(screen, LIGHT_CHECKER_BORDER, (x, y), WIDTH / 24, 5)
                    elif mark == 'O':
                        pygame.draw.circle(screen, LIGHT_CHECKER, (x, y), WIDTH / 24)
                        pygame.draw.circle(screen, LIGHT_CHECKER_BORDER, (x, y), WIDTH / 24, 5)
                        pygame.draw.circle(screen, RED_LIGHT, (x, y), WIDTH / 32, 10)
                    elif mark == 'X':
                        pygame.draw.circle(screen, DARK_CHECKER, (x, y), WIDTH / 24)
                        pygame.draw.circle(screen, DARK_CHECKER_BORDER, (x, y), WIDTH / 24, 5)
                        pygame.draw.circle(screen, RED_LIGHT, (x, y), WIDTH / 32, 10)

                        
    def highlight_actions(self, actions):
        # evidenzia le mosse possibili
        if self.engine_next_move:
            actions = [self.engine_next_move]
        for (action_type, (row, col), chain) in actions:
            if not self.selected_token:
                pos_left = col * (WIDTH / 8)
                pos_top = row * (HEIGHT / 8)
                pygame.draw.rect(screen, GREEN_DARK, ((pos_left, pos_top), (WIDTH / 8, HEIGHT / 8)))
            else:
                if self.selected_token[0] == row and self.selected_token[1] == col:
                    pos_left = col * (WIDTH / 8)
                    pos_top = row * (HEIGHT / 8)
                    pygame.draw.rect(screen, GREEN_DARK, ((pos_left, pos_top), (WIDTH / 8, HEIGHT / 8)))
                    if action_type == 'move':
                        (delta_row, delta_col) = chain
                        pos_left = (col + delta_col) * (WIDTH / 8)
                        pos_top = (row + delta_row) * (HEIGHT / 8)
                        pygame.draw.rect(screen, BLUE_DARK, ((pos_left, pos_top), (WIDTH / 8, HEIGHT / 8)))
                    elif action_type == 'jump':
                        prev_col = col
                        prev_row = row
                        for (delta_row, delta_col) in chain:
                            prev_col += delta_col
                            prev_row += delta_row
                            pos_left = prev_col * (WIDTH / 8)
                            pos_top = prev_row * (HEIGHT / 8)
                            pygame.draw.rect(screen, BLUE_DARK, ((pos_left, pos_top), (WIDTH / 8, HEIGHT / 8)))

Definiamo alcune funzioni ausiliarie per l'interfaccia grafica.

In [9]:
# restituisce la colonna cliccata
def get_clicked_column(mouse_pos):
    x = mouse_pos[0]
    for i in range(1, 8):
        if x < i * WIDTH / 8:
            return i - 1
    return 7


# restituisce la riga cliccata
def get_clicked_row(mouse_pos):
    y = mouse_pos[1]
    for i in range(1, 8):
        if y < i * HEIGHT / 8:
            return i - 1
    return 7

## Funzioni di gioco

Funzioni ausiliarie per il gioco.

In [10]:
# restituisce il giocatore che dovrà muovere
def opponent(player : str) -> str:
    if player == 'x':
        return 'o'
    else:
        return 'x'


# restiruisce il valore della cella identificata dalla coppia (row,column)
def cell(board, row : int, column : int):
    if row < 0 or row > 7 or column < 0 or column > 7:
        return None
    else:
        return board[row][column]

Funzione `can_move`: presi in ingresso una scacchiera (`board`), una coppia di valori interi che rappresentano la posizione iniziale (`start`) ed una coppia di valori interi che rappresentano la direzione di spostamento (`direction`) restituisce `True` se la pedina può essere spostata verso quella direzione (`False` altrimenti).

La posizione iniziale è una tupla `(row, column)` che rappresenta le coordinate sulla scacchiera della pedina. `direction` è una tupla `(delta_row, delta_column)` che sommata alla posizione iniziale fornisce le coordinate finali della pedina dopo la mossa.

In [11]:
def can_move(board, start, direction):
    row, col = start  # estraggo riga e colonna
    delta_row, delta_col = direction  # estraggo variazione di spostamento
    token = cell(board, row, col)  # estraggo il token che è presente nella cella
    if token == 'x' and delta_row < 0 or token == 'o' and delta_row > 0:  
        # le pedine nere (x) possono muoversi solo in avanti (delta_row > 0)
        # mentre le pedine bianche possono muoversi solo all'indietro (delta_row < 0)
        return False
    else:
        # restituisco True se la cella di arrivo è libera (identificata con '-'), altrimenti false
        # posso scrivere in maniera equivalente
        # t1 = cell(board, row+delta_row, col+delta_col)
        # if t1 == '-': 
        #     return True 
        # else: 
        #     return False
        return cell(board, row+delta_row, col+delta_col) == '-'

Funzione `can_jump`: presi in ingresso una scacchiera (`board`), una coppia di valori interi che rappresentano la posizione iniziale (`start`) ed una coppia di valori interi che rappresentano la direzione di spostamento (`direction`) restituisce `True` se la pedina può "saltare" verso quella direzione (`False` altrimenti). 

La differenza con la funzione precedente è che, in questo caso, non devo solamente verificare che la cella di destinazione sia libera ma anche che la pedina che sto "saltando" sia dell'avversario e sia "mangiabile". Ricordiamo che una pedina può "mangiare" una pedina avversaria ma non una dama.

In [12]:
def can_jump(board, start, direction):
    row, col = start  # estraggo riga e colonna corrente
    delta_row, delta_col = direction  # estraggo spostamento
    token = cell(board, row, col)  # estraggo il token che si trova nella posizione di arrivo
    if token == 'x' and delta_row < 0 or token == 'o' and delta_row > 0:
        # stesso controllo di can_move
        return False
    else:
        # estraggo il contenuto della cella che voglio saltare
        t = cell(board, row+int(delta_row/2), col+int(delta_col/2))
        
        if t is not None:
            if token.islower() and t.isupper():
                # con una pedina (token.islower() restituisce True se il Token è minuscolo, False altrimenti)
                # non posso mangiare una dama (t.isupper() restituisce True se t è maiuscolo, False altrimenti)
                return False
            else:
                # controllo di saltare una pedina avversaria (prima condizione) e che la cella
                # di arrivo sia vuota ('-')
                return t.lower() == opponent(token.lower()) and cell(board, row+delta_row, col+delta_col) == '-'

Funzione `jump`: presi in ingresso una scacchiera (`board`), una coppia di valori interi che rappresentano la posizione iniziale (`start`) ed una coppia di valori interi che rappresentano la direzione di spostamento (`direction`), restituisce una coppia di valori (tupla) che rappresentano la posizione di arrivo `(to_row, to_col)` e la nuova scacchiera aggiornata (`new_board`).

In [13]:
def jump(board, start, direction):
    row, col = start  # estraggo riga e colonna di partenza
    token = board[row][col]  # estraggo il token che si trova nella posizone corrente
    delta_row, delta_col = direction  # estraggo spostamento riga e colonna
    new_board = copy.deepcopy(board)  # copio la scacchiera: uso deepcopy perché non voglio perdere lo stato della scacchiera
    new_board[row][col] = '-'  # segno come libera la posizione nella quale mi trovo
    new_board[row+int(delta_row/2)][col+int(delta_col/2)] = '-'  # segno come libera la posizione che salto (mangio la pedina)
    to_row = row+delta_row  # riga di destinazione
    to_col = col+delta_col  # colonna di destinazione
    if token == 'x' and to_row == 7:  # se sono arrivato dall'altra parte della scacchiera per il nero
        token = 'X'  # dama nera
    if token == 'o' and to_row == 0:  # se sono arrivato dall'altra parte della scacchiera per il bianco
        token = 'O'  # dama bianca
    new_board[to_row][to_col] = token  # imposto il token nella posizione nella quale mi sposto
    return (to_row, to_col), new_board  # restituisco la nuova posizione e la scacchiera

Funzione `jumpingchains` (ricorsiva): dati in ingresso una scacchiera (`board`) e la posizione iniziale della pedina (`start`) restituisce una lista di salti (`chains`). Necessaria per mangiare più pedine.

In [14]:
def jumpingchains(board, start):
    chains = []  # inizialmente la catena di salti è vuota
    for dir in [(-2, -2), (-2, +2), (+2, -2), (+2, +2)]:  # controllo tutte le 4 possibili direzioni di spostamento ad 1 passo
        if can_jump(board, start, dir):  # se mi posso posso saltare (mangiare) nella direzione selezionata
            start1, board1 = jump(board, start, dir)  # salto nella direzione selezionata
            tailchains = jumpingchains(board1, start1)
            # chiamata ricorsiva della funzione: partendo dal nuovo stato (mi sono spostato perché ho mangiato una pedina)
            # richiamo la funzione stessa. In questo modo, la nuova chiamata ricorsiva mi dirà se ci sono ulteriori mangiate
            # possibili. Così facendo, posso controllare solamente i 4 spostamenti possibili ad un passo in ogni cliclo. 
            # Senza iterazione dovrei controllare tutti i possibili spostamenti (non solo quelli ad un passo) con un ciclo
            if tailchains == []:
                # se non ci sono mangiate multiple, restituisco solamente quelle ad un passo
                chains.append([dir])
            else:
                for tail in tailchains:
                    # per ogni mangiata multipla, restituisco la lista
                    chains.append([dir] + tail)
    return chains

Ciclo principale. Per far giocare il motore di gioco contro se stesso, togliere il commento alle linee che iniziano con `newevent = ` e quella successiva. La variabile `n_matches` controlla il numero di partite che vengono giocate. Durante il gioco è possibile far selezionare al motore la mossa migliore spingendo il tasto 'a'.

In [17]:
n_matches = 1
win_nero = 0
win_bianco = 0
draw = 0

for i in range(n_matches):
    pygame.init()
    size = (WIDTH, HEIGHT)
    screen = pygame.display.set_mode(size)

    ui = UI()

    # ciclo finché non viene chiusa la schermata
    done = False

    # velocità di aggiornamento
    clock = pygame.time.Clock()

    # ciclo
    while not done:
        # --- Main event loop
        # togliere il commento alle due linee seguenti per far giocare
        # il motore di gioco contro se stesso
        # newevent = pygame.event.Event(pygame.locals.KEYDOWN, unicode="a", key=pygame.locals.K_a, mod=pygame.locals.KMOD_NONE)  # create the event
        # pygame.event.post(newevent)
        for event in pygame.event.get():  # l'utente ha eseguito un comando
            if event.type == pygame.QUIT:  # click su chiudi
                done = True
            if not ui.is_game_over:
                if event.type == pygame.KEYDOWN:
                    if event.key == 97: # cliccata 'a'
                        ui.engine_move()
                if event.type == pygame.MOUSEBUTTONDOWN:
                    mouse_x, mouse_y = pygame.mouse.get_pos()
                    keys = pygame.key.get_pressed()
                    ui.evaluate_click((mouse_x, mouse_y), keys)
            else:
                done = True
                if ui.winner is None:
                    draw += 1
                elif ui.winner == 'x':
                    win_nero += 1
                else:
                    win_bianco += 1
                print("Nero:", win_nero, "Bianco:", win_bianco, "Draw:", draw)

        # riempio la scacchiera di nero
        screen.fill(BLACK_WOOD)

        # disegno lo stato
        ui.draw()

        # aggiorno la schermata
        pygame.display.flip()

        # limite 60 frame al secondo
        clock.tick(60)

    # chiusura finestra e quit
    pygame.quit()


('move', (5, 1), (-1, -1))
('move', (2, 4), (1, 1))
('move', (6, 0), (-1, 1))
('move', (3, 5), (1, 1))
('jump', (5, 7), [(-2, -2)])
